<a href="https://colab.research.google.com/github/KorStats/topicmodeling/blob/main/2_KoBERT_%EC%8B%A0%EB%AC%B8%EA%B8%B0%EC%82%AC_%EC%9B%90%EB%B3%B8(%EB%8B%A8%EB%AC%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 설치

In [ ]:
!pip install transformers datasets accelerate scikit-learn optuna sentencepiece

In [ ]:
!pip install transformers

In [ ]:

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch
import optuna
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import platform
import transformers
import joblib

In [ ]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# 사용할 데이터

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'   법률_train_original.csv   신문기사_train_original.csv   이전
'My Drive'	    법률_valid_original.csv   신문기사_valid_original.csv   졸업논문


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/신문기사_단문.csv', encoding='utf-8-sig')

In [ ]:
df['category'].value_counts()

,count
category,
종합,4000
경제,2000
사회,2000
정치,2000
스포츠,1000
"IT,과학",500
교육/입시/NIE,500


In [ ]:
data_list = []
for q, label in zip(df['text'], df['category'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(dataset_train)
test=pd.DataFrame(dataset_test)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

In [ ]:
train.shape

(9000, 2)

In [ ]:
test.shape

(3000, 2)

# 데이터 정제

In [ ]:
def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)

In [ ]:
train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)


Cleaning Special Characters: 100%|██████████| 3000/3000 [00:01<00:00, 1782.98it/s]


In [ ]:
# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# KoBERT

In [ ]:
# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

In [ ]:
# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 최적의 하이퍼파라미터 찾기

In [ ]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 샘플링
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)

    # TrainingArguments 설정
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        logging_steps=10,
        per_device_train_batch_size=batch_size,
        num_train_epochs=3,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        report_to="none"
    )

    # Trainer 생성
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1"]


# Optuna 탐색
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=5)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", study.best_params)

[I 2025-01-27 02:42:27,921] A new study created in memory with name: no-name-11170838-2946-4569-b392-255063aafe70
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,1.722400,1.749288,0.070352
2,1.702100,1.742735,0.070352
3,1.810200,1.741097,0.070352


[I 2025-01-27 03:05:44,098] Trial 0 finished with value: 0.07035175879396985 and parameters: {'learning_rate': 0.0001905984058999072, 'batch_size': 32, 'weight_decay': 0.25850430647724326}. Best is trial 0 with value: 0.07035175879396985.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to b

Epoch,Training Loss,Validation Loss,F1
1,0.981800,0.950967,0.645427
2,0.748000,0.801380,0.711705
3,0.534700,0.765764,0.723614


[I 2025-01-27 03:29:30,968] Trial 1 finished with value: 0.7236141341015828 and parameters: {'learning_rate': 5.962275064510067e-05, 'batch_size': 64, 'weight_decay': 0.05499685060385599}. Best is trial 1 with value: 0.7236141341015828.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be 

...

Epoch,Training Loss,Validation Loss,F1
1,0.800100,0.908711,0.661578
2,0.778800,0.753752,0.727797
3,0.403700,0.738188,0.742859


[I 2025-01-27 04:19:20,824] Trial 3 finished with value: 0.7428586799466663 and parameters: {'learning_rate': 4.33035913885986e-05, 'batch_size': 16, 'weight_decay': 0.02274003262981744}. Best is trial 3 with value: 0.7428586799466663.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be a

Epoch,Training Loss,Validation Loss,F1
1,1.120800,1.109729,0.494150
2,0.890400,0.933711,0.644284
3,0.812700,0.912761,0.646775


[I 2025-01-27 04:44:38,221] Trial 4 finished with value: 0.6467751404529114 and parameters: {'learning_rate': 2.0762119749142177e-05, 'batch_size': 64, 'weight_decay': 0.03379293050441634}. Best is trial 3 with value: 0.7428586799466663.


Best Hyperparameters: {'learning_rate': 4.33035913885986e-05, 'batch_size': 16, 'weight_decay': 0.02274003262981744}


Best Hyperparameters: {'learning_rate': 4.33035913885986e-05, 'batch_size': 16, 'weight_decay': 0.02274003262981744

# 최적의 하이퍼파라미터로 재학습

In [ ]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [ ]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Best Hyperparameters: {'learning_rate': 4.33035913885986e-05, 'batch_size': 16, 'weight_decay': 0.02274003262981744

In [ ]:
# 7. 최적의 파라미터로 학습 설정
# 최적의 하이퍼파라미터를 찾은 후, training_args를 설정합니다.
training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=3,  # 에포크 수 고정
    per_device_train_batch_size=16,  # 배치 크기 고정
    learning_rate=4.33035913885986e-05,  # 최적화된 학습률
    weight_decay=0.02274003262981744,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    disable_tqdm=False,
    load_best_model_at_end=True,
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

Epoch,Training Loss,Validation Loss,F1
1,0.708400,0.903765,0.662619
2,0.702600,0.745139,0.721167
3,0.436200,0.759436,0.739004


Elapsed Time: 418.00 seconds
Average CPU Usage: 58.95%


In [ ]:
# 모델 저장
joblib.dump(trainer7, '/content/gdrive/MyDrive/trainer_news_text.pkl')

print("모델이 저장되었습니다.")

모델이 저장되었습니다.


In [ ]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

loaded_model = joblib.load('/content/gdrive/MyDrive/trainer7.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = loaded_model.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

In [ ]:
test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]
selected_columns7.head(10)

,text,category_prediction
0,국세청 적발 탈세 등 악용 우려 세종충청일보 장중식 기자 신용카드 위장가맹점 적발 ...,사회
1,한전 정부혁신박람회 이용환 기자 한전이 미세먼지를 청소하는 수소버스로 제 대한민국 ...,경제
2,이데일리 한광범 기자 브로드밴드는 텔레콤 와 함께 랜케이블망에서도 사용자에게 상하...,"IT,과학"
3,조배숙 의원 예산정책협 촉구 착공 뒤 착공 김형민 기자 전북 국회의원전라북도 예산정...,정치
4,교육청과 학교가 나서 현실적 대책 마련해야 홍성장 기자 광주지역 초등학교병설유치원에...,교육/입시/NIE
5,카드사의 수수료율 인상 방침에 대형 유통업계가 마케팅 축소를 검토하고 있다 이데일리...,경제
6,태광그룹 계열 한국케이블텔레콤 대표 신동선이 우체국에서 알뜰폰 판매를 시작했다 의 ...,경제
7,박길수 기자 폭염 속 농업인에게 큰 도움 오는 수확작업 대행 핵심공약 축산농가 경쟁...,종합
8,이홍재 기자 한강유역환경청은 환경부가 전국 지자체를 대상으로 실시한 공공하수도의 운...,종합
9,삼성전자가 개척한 글로벌 폴더블폰 시장에 화웨이모토로라가 가세하면서 한중 맞대결이 ...,"IT,과학"


In [ ]:
selected_columns7